In [83]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os, sys

from typing import Tuple, Dict, List
from pydantic import BaseModel, Field
import sklearn
import statsmodels
import openai

In [167]:
TASK1_SUBMIT_DF = pd.read_csv("子赛题1：机组数据驱动的风电场短期风况预测/filtered_初赛_提交模板.csv")
agg_df = []
for keys, df in TASK1_SUBMIT_DF.groupby(["风场", "风机"]):
    paths = glob.glob("子赛题1：机组数据驱动的风电场短期风况预测/训练集/" + keys[0] + "/" + keys[1] + "/*")
    paths.sort()

    for path in paths[::-1]:
        df = pd.read_csv(path)
        df = df.dropna()
        if len(df) == 0:
            continue
        
        agg_df.append(list(keys) + [df["风向"].dropna().iloc[-1], df["风速"].dropna().iloc[-1]])
        break

agg_df = pd.DataFrame(agg_df)
agg_df.columns = ["风场", "风机", "风向", "风速"]
pd.merge(TASK1_SUBMIT_DF.drop(["风向", "风速"], axis=1), agg_df, on=["风场", "风机"], how="left").to_csv("task1_sub.csv", index=None)

In [168]:
columns = ['SeqNum']
for i in range(1, 57):
    columns.append(f'Prediction{i}')

data = []
for i in range(1, 4):
    row = [i] + [0] * 56
    data.append(row)

TASK2_SUBMIT_DF = pd.DataFrame(data, columns=columns)

job1 = pd.read_excel("子赛题2：水电站入库流量预测/测试集_初赛/预测01/入库流量数据.xlsx")
TASK2_SUBMIT_DF.loc[0] = [1] + [job1["Qi"].tail(30).mean()] * 56

job2 = pd.read_excel("子赛题2：水电站入库流量预测/测试集_初赛/预测02/入库流量数据.xlsx")
TASK2_SUBMIT_DF.loc[1] = [2] + [job2["Qi"].tail(30).mean()] * 56

job3 = pd.read_excel("子赛题2：水电站入库流量预测/测试集_初赛/预测03/水库流量数据.xlsx")
TASK2_SUBMIT_DF.loc[2] = [3] + [job3["Qi"].tail(30).mean()] * 56
TASK2_SUBMIT_DF.to_csv("task2_sub.csv", index=None)

In [169]:
task_kucun = pd.read_csv("./子赛题3：离散制造行业产品物料需求智能预测/训练集/历史库存量.csv")
task_xuqiu = pd.read_csv("./子赛题3：离散制造行业产品物料需求智能预测/训练集/历史需求量.csv")
task_xuqiu["物料编码"] = task_xuqiu["物料编码"].astype(int)

sub = task_xuqiu.groupby(["工厂编码", "物料编码"])["需求量"].mean().reset_index()
sub.columns = ["工厂编码", "物料编码", "M+1月预测需求量"]
sub["M+2月预测需求量"] = sub["M+1月预测需求量"]
sub.to_csv("task3_sub.csv", index=None)